In [7]:
import pandas as pd
import json
import os
import requests
from datetime import datetime, time

In [ ]:

file1 = 'rawdata/불법주정차 신고현황(21년9월29일_22년10월31일).csv'
file2 = 'rawdata/불법주정차 신고현황(22.11월~23.10월).csv'
file3 = 'rawdata/불법주정차 신고현황(23년11월1일_24년3월13일).csv'

df_reports_1 = pd.read_csv(file1, encoding='utf-8')
df_reports_2 = pd.read_csv(file2, encoding='utf-8')
df_reports_3 = pd.read_csv(file3, encoding='utf-8')


In [6]:
display(df_reports_1.head())

,민원접수일,민원접수시간,주소,경도,위도
0,20210929,00:05:00,서울 강서구 화곡로66길 130,126.859911,37.555537
1,20210929,00:38:00,중구 다산로48길 33 단우물어린이집앞,127.018013,37.567483
2,20210929,00:38:00,서울특별시 중구 마장로 19,127.012793,37.569183
3,20210929,00:50:00,서울특별시 강서구 까치산로15길 21,126.847125,37.547040
4,20210929,01:15:00,서울특별시 중구 다산로 293,127.015703,37.569085


In [4]:
file4 = 'rawdata/서울시 공영주차장 안내 정보.csv'
df_park_info = pd.read_csv(file4, encoding='cp949')

In [8]:
df_reports = pd.concat([df_reports_3, df_reports_1, df_reports_2], axis=0)


In [5]:
display(df_park_info.columns)

Index(['주차장코드', '주차장명', '주소', '주차장 종류', '주차장 종류명', '운영구분', '운영구분명', '전화번호',
       '주차현황 정보 제공여부', '주차현황 정보 제공여부명', '총 주차면', '유무료구분', '유무료구분명', '야간무료개방여부',
       '야간무료개방여부명', '평일 운영 시작시각(HHMM)', '평일 운영 종료시각(HHMM)', '주말 운영 시작시각(HHMM)',
       '주말 운영 종료시각(HHMM)', '공휴일 운영 시작시각(HHMM)', '공휴일 운영 종료시각(HHMM)',
       '최종데이터 동기화 시간', '토요일 유,무료 구분', '토요일 유,무료 구분명', '공휴일 유,무료 구분',
       '공휴일 유,무료 구분명', '월 정기권 금액', '노상 주차장 관리그룹번호', '기본 주차 요금',
       '기본 주차 시간(분 단위)', '추가 단위 요금', '추가 단위 시간(분 단위)', '버스 기본 주차 요금',
       '버스 기본 주차 시간(분 단위)', '버스 추가 단위 시간(분 단위)', '버스 추가 단위 요금', '일 최대 요금',
       '위도', '경도'],
      dtype='object')

In [41]:

# Rename columns
df_park_info = df_park_info.rename(columns={
    '총 주차면': '총주차면',
    '유무료구분': '평일유료',
    '평일 운영 시작시각(HHMM)': '평일시작',
    '평일 운영 종료시각(HHMM)': '평일종료',
    '주말 운영 시작시각(HHMM)': '토요일시작',
    '주말 운영 종료시각(HHMM)': '토요일종료',
    '공휴일 운영 시작시각(HHMM)': '공휴일시작',
    '공휴일 운영 종료시각(HHMM)': '공휴일종료',
    '토요일 유,무료 구분': '토요일유료',
    '공휴일 유,무료 구분': '공휴일유료'
})

# Define the desired column order
new_columns = ['주소', '총주차면', '평일유료', '토요일유료', '공휴일유료', '평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료', '위도', '경도']

# Reorder the columns in the DataFrame
df_parks = df_park_info[new_columns]

display(df_parks.head())
# display(df_parks.tail())


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,900,1900,900,1900,900,1900,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,0,2400,0,2400,0,2400,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN


In [42]:
def parse_time(value):
    """Convert an integer or string time (HHMM or HH:MM:SS) to a datetime.time object."""
    if pd.isna(value):
        return None

    # build a zero-padded string
    time_str = str(int(value)).zfill(4) if isinstance(value, (int, float)) else str(value)

    # special-case “2400” or “24:00[:00]” as midnight
    if time_str in ("2400", "24:00", "24:00:00"):
        return time(0, 0)

    try:
        # Try parsing as HHMM
        return pd.to_datetime(time_str, format="%H%M").time()
    except ValueError:
        # Fallback to general parser (HH:MM:SS etc.)
        try:
            return pd.to_datetime(time_str).time()
        except Exception:
            return None

In [43]:
time_columns = ['평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료']
for col in time_columns:
    df_parks[col] = df_parks[col].apply(parse_time).copy()

display(df_parks.head())

/var/folders/pf/y0k140gn11d95ybm0dgtk6m80000gn/T/ipykernel_8560/50185423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parks[col] = df_parks[col].apply(parse_time).copy()


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [46]:
# Find rows with missing values in columns except '위도' or '경도'
missing_rows = df_parks[df_parks.drop(columns=['위도', '경도']).isnull().any(axis=1)]

# Display rows with missing values
display(missing_rows)

# Drop rows with missing values in columns except '위도' or '경도'
df_parks_dropped = df_parks.drop(missing_rows.index)

# Display the updated DataFrame
display(df_parks_dropped)

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
7688,종로구 명륜4가 105-3,NaN,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8058,용산구 남영동 71-2,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8059,용산구 남영동 71-2,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8073,송파구 가락동 156-11,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8080,송파구 문정동 640-9,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8278,송파구 풍납동 306-25,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8279,송파구 풍납동 222-75,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8280,송파구 풍납동 292-33,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8281,송파구 풍납동 140-3,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8505,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8506,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8507,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,NaN,NaN
8508,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [48]:
df_parks_dropped = df_parks_dropped.reset_index(drop=True)
# Display the updated DataFrame
display(df_parks_dropped)


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8327,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8328,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,NaN,NaN
8329,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [50]:
df_parks_missing = df_parks_dropped[df_parks_dropped.isnull().any(axis=1)][['주소']].copy()
df_parks_missing = df_parks_missing.drop_duplicates().reset_index(drop=True)
display(df_parks_missing.head())

,주소
0,도봉구 창동 24-0
1,노원구 상계동 770-2
2,성동구 마장동 463-2
3,영등포구 당산동3가 385-0
4,영등포구 대림동 780-0


In [51]:
# Function to get coordinates for a given address
def get_coordinates(address):
    apiurl = "https://api.vworld.kr/req/address?"
    apikey =  "D9C1CFC9-0FD3-3304-A3E7-131C33A91190"

    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",
        "key": apikey
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            point_info = data['response']['result']['point']
            return point_info['x'], point_info['y']
    return None, None


In [ ]:
# Apply the function to the '주소' column and store the results
df_parks_missing[['위도', '경도']] = df_parks_missing['주소'].apply(lambda addr: pd.Series(get_coordinates(addr)))

# Display the updated dataframe
df_parks_missing.head()

,주소,위도,경도
0,도봉구 창동 24-0,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,127.03510376366461,37.56996805245046
3,영등포구 당산동3가 385-0,126.89577762856109,37.52554507178421
4,영등포구 대림동 780-0,126.89444474098582,37.49923824170827


In [54]:
df_parks_missing

,주소,위도,경도
0,도봉구 창동 24-0,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,127.03510376366461,37.56996805245046
3,영등포구 당산동3가 385-0,126.89577762856109,37.52554507178421
4,영등포구 대림동 780-0,126.89444474098582,37.49923824170827
...,...,...,...
1054,강북구 미아동 791-1360,127.01337142398819,37.626166339528766
1055,강북구 미아동 791-1429,127.01571664763247,37.62589357898619
1056,강북구 미아동 748-2,127.01847091587678,37.622737640361635
1057,구로구 구로동 1-4,126.89159944589176,37.50731316416754


In [ ]:
# Set '주소' as index for both DataFrames for alignment
df_parks_dropped.set_index('주소', inplace=True)
df_parks_missing.set_index('주소', inplace=True)

# Update '위도' and '경도' values in df_parks_dropped using the data from df_parks_missing
df_parks_dropped.update(df_parks_missing[['위도', '경도']])

# Restore '주소' as a column
df_parks_dropped.reset_index(inplace=True)

display(df_parks_dropped.head())